In [30]:
import numpy as np 
from scipy.stats import norm

class OptionsPricing():
    
    """
    Preface:
        The classic Black Scholes formula as implemented below assumes:
            1) The amount of time that is used for the determining the distribution of the underlying value 
            at expiry is the same amount of time that should be used for discounting the forward price of 
            the option.  Frequently these two amounts of time are different as options rarely settle at expiry
            but instead settle at expiry plus a few days.
            2) The interest rate used to project the underlying value at expiry based on the underlying spot 
            value is the same as the interest rate that should be used for discounting the forward price of
            the option.  In crytpo markets in 2025, for example, these two rates are very different.
        To overcome these shortcomings, the following implementation is recommended:
            1) Set the interest rate in the Black Scholes formula to zero.
            2) Calculate the underlying value at expiry prior to calling the formula.  Enter the underlying
            value at epxiry as the undelrying_value variable.
            3) Calculate the amount of time to expiration of the option prior to calling the formual.  Set
            the time variable equal to this amount.
            4) Discount the formulas's resulting forward price of the option after using the formula below
            using the Time_Value_Money Class and choices for rate and time that are specific to the discounting
            situation (time to payment rather than time to expiry and rate for discounting rather than forward
            rate of underlying).
        The term "value" is used rather than "price" to emphasize that formulas below can be used for rates, too
    
    Parameters:
        call_or_put (string): either 'call' or 'put'
        underlying_value (float): see note above, the value of the underlying
        time (float): see note above, fraction of a year (use Daycount_Math prior to calling functions below)
        strike_value (float): same units as underlying_value
        volatility (float): percentage of underlying value (should be the same value regardless of using 
            forward or spot underlying price)
        interest_rate (float): see note above, continuously compounded rate used to (i) discount forward option 
            value to present value and (ii) create underlying value at expiry (if spot underlying value is used
            as the input) 
    """
        
    def __init__(self):
        # constructor 
        pass
    

    def bs_price_euro(self, 
                      put_or_call, 
                      underlying_value, #see note above, best to set equal to expected value at expiry
                      strike_value,
                      time, # see note above, best to set equal to time to option expiration
                      volatility,
                      interest_rate=0 # see note above, discount option price outside of formula
                     ):
        
        # Convert all inputs to NumPy arrays for broadcasting
        put_or_call      = np.asarray(put_or_call, dtype=str)
        underlying_value = np.asarray(underlying_value, dtype=float)
        strike_value     = np.asarray(strike_value, dtype=float)
        time             = np.asarray(time, dtype=float)
        volatility       = np.asarray(volatility, dtype=float)
        interest_rate    = np.asarray(interest_rate, dtype=float)
                   
        # If put_or_call is a scalar string, broadcast it to match inputs
        if np.isscalar(put_or_call):
            put_or_call = np.full_like(underlying_value, put_or_call, dtype=object)
                   
        # Validate input parameter
        if not np.all(np.isin(put_or_call, ['put', 'call'])):
            raise ValueError("put_or_call must contain only 'put' or 'call'")
    
        # Compute present value discount factor
        pv_factor = np.exp(interest_rate * time) #if interest_rate == 0 then pv_factor will equal one
        
        # Compute d1 and d2
        d1, d2 = self.calc_bs_d(underlying_value, strike_value, time, volatility, interest_rate)
        
        # Create +1 for calls, -1 for puts
        put_call_scalar = np.where(put_or_call == 'call', 1, -1)
                
        # Compute N(d1) and N(d2)
        nd1 = norm.cdf(d1 * put_call_scalar)
        nd2 = norm.cdf(d2 * put_call_scalar)
        
        # Compute final option price
        underlying_product =  put_call_scalar * nd1 * underlying_value 
        strike_product     = -put_call_scalar * nd2 * (strike_value * pv_factor)
        
        return underlying_product + strike_product
       
        
    def bs_d_both(self, underlying_value, strike_value, time, volatility, interest_rate=0):
        # Calcs d1 and d2 for an option
                
        # Convert inputs to NumPy arrays for elementwise math
        underlying_value = np.asarray(underlying_value, dtype=float)
        strike_value     = np.asarray(strike_value, dtype=float)
        time             = np.asarray(time, dtype=float)
        volatility       = np.asarray(volatility, dtype=float)
        interest_rate    = np.asarray(interest_rate, dtype=float)
        
        # Prevent division by zero or invalid sqrt(time)
        time = np.maximum(time, 1e-12)
        volatility = np.maximum(volatility, 1e-12)

        # Core Black–Scholes d1 & d2 math
        vol_sqrt_time = volatility * np.sqrt(time)     
        log_moneyness = np.log(underlying_value / strike_value)
        growth_rate = interest_rate * time # if interest_rate == 0 then growth_rate will equal one     
        d0 = (log_moneyness + growth_rate) / vol_sqrt_time
        
        # Calc d1 & d2
        d1 = d0 + vol_sqrt_time / 2.0
        d2 = d0 - vol_sqrt_time / 2.0
        
        return d1, d2
    
    
    def intrinsic_value(underlying_value, strike_value_pv):
        # Calcs intrinsic values for options
        # Subtract intrinsic values from options prices to get time values
        # Calc the present value of the strike values prior to calling this function
        
        underlying_value = np.asarray(underlying_value, dtype=float)
        strike_value_pv  = np.asarray(strike_value_pv, dtype=float)
        
        call_intrinsic = np.maximum(underlying_value - strike_value_pv, 0.0)
        put_intrinsic  = np.maximum(strike_value_pv - underlying_value, 0.0)
        
        return call_intrinsic, put_intrinsic
    